In [1]:
import os
import sys

# Import the planner components from the provided lab9.py file
# Note: Ensure lab9.py is in the same directory as this notebook
from lab9 import (
    validate_user_conditions,
    create_custom_problem,
    PDDLParser,
    ActionGrounder,
    bfs_search,
    astar_search, # NOTE by Claude!
    print_plan_execution,
    Predicate,
    State,
    Action,
    TOOLS
)

print("✅ Environment set up successfully. Planner components loaded.")

✅ Environment set up successfully. Planner components loaded.


# Lab 9: Classical Planning with PDDL and Breadth-First Search

## Overview
In this lab, we will explore the fundamental components of **Automated Planning**:
1.  **State Representation:** Defining the world using PDDL (Planning Domain Definition Language) predicates.
2.  **Action Schemas:** Understanding how actions (like `pick-up`, `move`, `slice`) transform the state.
3.  **Search Algorithms:** Using **Breadth-First Search (BFS)** to find a sequence of actions that leads from an *Initial State* to a *Goal State*.

## The Domain: CIFAR-100 Process
We are working in a specific domain environment consisting of:
* **Locations:** A `lab` and `outdoors`.
* **Objects:** 100 different items from the CIFAR-100 dataset (e.g., `apple`, `bear`, `couch`) and 3 specific tools (`knife`, `sponge`, `dslr`).
* **The Agent:** A robot that can move between rooms, pick up items, and apply tools to objects (e.g., using a knife to slice an apple).

## The Algorithm: Breadth-First Search (BFS)
The planner in this notebook uses **BFS** to find solutions.
* **The Good:** BFS is **complete** and **optimal** for unweighted graphs. It guarantees finding the solution with the fewest number of steps (shortest plan).
* **The Bad:** BFS is computationally expensive. As the plan length increases, the number of states to explore grows exponentially.

> **⚠️ Note for Coursework:** While BFS is excellent for understanding the mechanics of state-space search, it is inefficient for complex problems. In **Coursework 2 (CW2)**, you will be expected to implement more sophisticated search algorithms (like A*) to handle larger search spaces.

## Notebook Structure
1.  **Setup:** Load the PDDL parser and domain files.
2.  **Sanity Checks:** Verify that our domain and problem files are parsed correctly.
3.  **Plan Generation:** We will define custom scenarios (Initial Conditions + Goal) and run the planner to generate an execution trace.

In [2]:
DOMAIN_FILE = "domain.pddl"
PROBLEM_FILE = "problem.pddl"

# 1. Parse the Domain and Problem files
print(f"Loading files...\n  - {DOMAIN_FILE}\n  - {PROBLEM_FILE}\n")
actions = PDDLParser.parse_domain(DOMAIN_FILE)
objs, init, goal = PDDLParser.parse_problem(PROBLEM_FILE)

# 2. Validate Domain Actions
print(f"✓ Domain: {len(actions)} actions found.")

# 3. Validate Problem Objects
# We distinguish between the 100 CIFAR objects and the 3 Tools (knife, sponge, dslr)
cifar_count = len(objs.get('item', set()) - TOOLS)
tool_count = len(objs.get('tool', set()))

print(f"✓ Problem: {cifar_count} CIFAR items + {tool_count} Tools.")

# 4. Validate Initial State
print(f"✓ Initial state: {len(init.predicates)} predicates.")

Loading files...
  - domain.pddl
  - problem.pddl

✓ Domain: 7 actions found.
✓ Problem: 100 CIFAR items + 2 Tools.
✓ Initial state: 349 predicates.


In [3]:
print("--- TEST: STATE TRANSITIONS ---")

# 1. Create a concrete Initial State using real Predicate objects
#    Scenario: An apple is on the floor, and our hand is empty.
p_at = Predicate.from_string("(at apple lab)")
p_empty = Predicate.from_string("(hand-empty)")
initial_state = State(frozenset([p_at, p_empty]))
print(p_at)
print(p_empty)
print(initial_state)

initial_state = State(frozenset([p_at, p_empty]))

# 2. Manually construct a Grounded Action
#    Action: pick-up(apple, lab)
#    (We manually set the effects here to verify the State logic)
action = Action(
    name="pick-up",
    parameters=("apple", "lab"),
    preconditions=frozenset([p_at, p_empty]),
    add_effects=frozenset([Predicate.from_string("(holding apple)")]),
    del_effects=frozenset([p_at, p_empty]) # These should disappear
)

print(f"1. BEFORE: {sorted([str(p) for p in initial_state.predicates])}")
print(f"2. ACTION: {action}")


# 3. CALL YOUR CODE
new_state = initial_state.apply_action(action)

print(f"3. AFTER : {sorted([str(p) for p in new_state.predicates])}")

# Verification
assert Predicate.from_string("(holding apple)") in new_state.predicates
assert Predicate.from_string("(at apple lab)") not in new_state.predicates
print("\n✓ SUCCESS: The apple was successfully transferred from floor to hand.")

--- TEST: STATE TRANSITIONS ---
(at apple lab)
(hand-empty)
State(predicates=frozenset({Predicate(name='at', args=('apple', 'lab')), Predicate(name='hand-empty', args=())}))
1. BEFORE: ['(at apple lab)', '(hand-empty)']
2. ACTION: (pick-up apple lab)
3. AFTER : ['(holding apple)']

✓ SUCCESS: The apple was successfully transferred from floor to hand.


In [4]:
print("--- TEST: ACTION GROUNDING ---")

# 1. Define a Schema (Template) with variables
#    Schema: drop(?item)
schema = Action(
    name="drop",
    parameters=("?item",),
    preconditions=frozenset([Predicate.from_string("(holding ?item)")]),
    add_effects=frozenset([Predicate.from_string("(empty-hand)")]),
    del_effects=frozenset([Predicate.from_string("(holding ?item)")])
)

# 2. Define the Bindings (The Instruction)
#    "Replace ?item with 'frog'"
bindings = {"?item": "frog"}

print(f"TEMPLATE: {schema}")
print(f"BINDINGS: {bindings}")

# 3. CALL THE STUDENT'S CODE
grounded_action = schema.instantiate(bindings)

print(f"RESULT  : {grounded_action}")

# Verification
# We check if the internal sets were actually updated
precond_str = str(list(grounded_action.preconditions)[0])
print(f"PRECOND : {precond_str}")

assert "frog" in precond_str
assert "?item" not in precond_str
print("\n✓ SUCCESS: Variable '?item' was correctly replaced by 'frog'.")

--- TEST: ACTION GROUNDING ---
TEMPLATE: (drop ?item)
BINDINGS: {'?item': 'frog'}
RESULT  : (drop frog)
PRECOND : (holding frog)

✓ SUCCESS: Variable '?item' was correctly replaced by 'frog'.


In [5]:
print("--- TEST: BFS ALGORITHM (TOY PROBLEM) ---")

# 1. Setup a tiny universe
#    Start: (at-A)
#    Goal:  (at-B)
start_pred = Predicate.from_string("(at-A)")
end_pred   = Predicate.from_string("(at-B)")

init_state = State(frozenset([start_pred]))
goal_preds = frozenset([end_pred])

# 2. Define the only possible move
#    Action: move-A-B
move_action = Action(
    name="move",
    parameters=("A", "B"),
    preconditions=frozenset([start_pred]),
    add_effects=frozenset([end_pred]),
    del_effects=frozenset([start_pred])
)

print(f"INIT: {init_state}")
print(f"GOAL: {goal_preds}")

# 3. CALL THE STUDENT'S CODE
#    We pass a list containing just our one move
plan = bfs_search(init_state, goal_preds, [move_action], verbose=True)

if plan:
    print(f"\nPLAN FOUND: {[str(a) for a in plan]}")
    assert len(plan) == 1
    assert plan[0].name == "move"
    print("✓ SUCCESS: BFS found the path from A to B.")
else:
    print("✗ FAILURE: BFS returned None.")

--- TEST: BFS ALGORITHM (TOY PROBLEM) ---
INIT: State(predicates=frozenset({Predicate(name='at-A', args=())}))
GOAL: frozenset({Predicate(name='at-B', args=())})

BFS search starting
Found plan! Iterations: 2, Plan length (Depth): 1

PLAN FOUND: ['(move A B)']
✓ SUCCESS: BFS found the path from A to B.


## The Plan Generator

To streamline our experiments, we will use a helper function called `plan_generator`. This function abstracts away the file management and PDDL parsing steps, allowing you to focus on defining the **Initial State** and the **Goal**.

**What this function does:**
1.  **Validation:** It strictly checks your input predicates against the Domain vocabulary (to catch typos!).
2.  **Problem Creation:** It generates a temporary `problem.pddl` problem file on the fly, reflecting the input begining and  goal states.
3.  **Grounding & Search:** It generates all possible specific actions and runs **BFS** (inefficient!) to find a solution.
4.  **Trace Generation:** It returns a step-by-step list of actions and state changes.

In [6]:
def plan_generator(obj_name,
                  initial_state,
                  goal_state):
    """
    Wraps the PDDL planning process: Validation -> Problem Gen -> Search -> Trace.
    """

    # Ensure inputs are sets
    initial_state = set(initial_state)
    goal_state = set(goal_state)

    # 1. Validate Inputs (Fail fast if there are typos)
    validate_user_conditions(initial_state)
    validate_user_conditions(goal_state)

    # 2. Create Temporary Problem File
    # We pass the user's specific initial/goal states to override the defaults
    temp_prob = create_custom_problem(PROBLEM_FILE, initial_state, goal_state, obj_name)

    try:
        # 3. Parse Domain and the new Problem file
        actions_dict = PDDLParser.parse_domain(DOMAIN_FILE)
        objs, init, goal = PDDLParser.parse_problem(temp_prob)

        # 4. Ground Actions (Generate specific actions like 'pick-up apple')
        grounder = ActionGrounder(actions_dict, objs)
        grounded = grounder.ground_all()

        # 5. Run Search
        # NOTE: This is very inefficient. You will need to do better than BFS in CW2.
        plan = astar_search(init, goal, grounded, verbose=True) #NOTE just changed this to astar search!

        if plan is None:
            return None

        # 6. Build Execution Trace
        changes, curr_state = [], init
        for i, action in enumerate(plan, 1):
            curr_state = curr_state.apply_action(action)
            changes.append({
                'step': i,
                'action': str(action),
                'added': sorted(map(str, action.add_effects)),
                'removed': sorted(map(str, action.del_effects))
            })

        return changes

    finally:
        # 7. Cleanup (Delete temporary file)
        if os.path.exists(temp_prob):
            os.unlink(temp_prob)

In [7]:
# --- SCENARIO 1: The Prepared Chef ---
test_name = "Instant Slice (Depth 1)"
obj_name = "apple"

# State: Robot is ready. It has the knife and is near the apple.
initial_state = {
    "(agent-at lab)",
    "(holding knife)",
    "(at apple lab)",
    "(whole apple)",
    "(clear apple)"
}

goal_state = {
    "(cut-into-pieces apple)"
}

print(f"\n{'='*80}\nScenario: {test_name}\n{'='*80}")
changes = plan_generator(obj_name, initial_state, goal_state)
if changes:
    print_plan_execution(changes, obj_name, initial_state)
    print(f"✓ PASS: {test_name}")
else:
    print(f"✗ FAIL: {test_name}")


Scenario: Instant Slice (Depth 1)

A* search starting
Found plan! Iterations: 2, Plan length: 1

📋 EXECUTION TRACE: APPLE
STEP   | ACTION                              | STATE CHANGES / INITIAL STATE           
------------------------------------------------------------------------------------------
0      | (INITIAL STATE)                     | (agent-at lab), (at apple lab), (clear apple), (holding knife), (whole apple)
1      | (slice-object apple lab)            | ++ (cut-into-pieces apple) | -- (whole apple)
------------------------------------------------------------------------------------------
✓ GOAL ACHIEVED

✓ PASS: Instant Slice (Depth 1)


In [8]:
test_name = "Basic Slice (Depth 2)"
obj_name = "pear"

# Context: The robot is in the lab with the pear and the knife,
# but it forgot to pick up the knife first!
initial_state = {
    "(agent-at lab)",
    "(hand-empty)",
    "(at pear lab)",
    "(whole pear)",
    "(clear pear)",
    "(at knife lab)",
    "(clear knife)"
}

goal_state = {
    "(cut-into-pieces pear)"
}

print(f"\n{'='*80}\nScenario: {test_name}\n{'='*80}")

changes = plan_generator(obj_name, initial_state, goal_state)

if changes:
    print_plan_execution(changes, obj_name, initial_state)
    print(f"✓ PASS: {test_name}")
else:
    print(f"✗ FAIL: {test_name}")


Scenario: Basic Slice (Depth 2)

A* search starting
Found plan! Iterations: 3, Plan length: 2

📋 EXECUTION TRACE: PEAR
STEP   | ACTION                              | STATE CHANGES / INITIAL STATE           
------------------------------------------------------------------------------------------
0      | (INITIAL STATE)                     | (agent-at lab), (at knife lab), (at pear lab), (clear knife), (clear pear), (hand-empty), (whole pear)
1      | (pick-up knife lab)                 | ++ (holding knife) | -- (at knife lab), (hand-empty)
2      | (slice-object pear lab)             | ++ (cut-into-pieces pear) | -- (whole pear)
------------------------------------------------------------------------------------------
✓ GOAL ACHIEVED

✓ PASS: Basic Slice (Depth 2)


In [9]:
test_name = "The Commuter (Depth 2)"
obj_name = "beetle"

# Context: The agent has the camera, but is in the wrong room!
initial_state = {
    "(agent-at outdoors)",
    "(holding dslr)",
    "(at beetle lab)",
    "(clear beetle)"
}

goal_state = {
    "(documented beetle)"
}

print(f"\n{'='*80}\nScenario: {test_name}\n{'='*80}")

changes = plan_generator(obj_name, initial_state, goal_state)

if changes:
    print_plan_execution(changes, obj_name, initial_state)
    print(f"✓ PASS: {test_name}")
else:
    print(f"✗ FAIL: {test_name}")


Scenario: The Commuter (Depth 2)

A* search starting
Found plan! Iterations: 3, Plan length: 2

📋 EXECUTION TRACE: BEETLE
STEP   | ACTION                              | STATE CHANGES / INITIAL STATE           
------------------------------------------------------------------------------------------
0      | (INITIAL STATE)                     | (agent-at outdoors), (at beetle lab), (clear beetle), (holding dslr)
1      | (walk-between-rooms outdoors lab)   | ++ (agent-at lab) | -- (agent-at outdoors)
2      | (take-photo beetle lab)             | ++ (documented beetle)
------------------------------------------------------------------------------------------
✓ GOAL ACHIEVED

✓ PASS: The Commuter (Depth 2)


In [10]:
test_name = "The Hungry Architect (Depth 3)"
obj_name = "sweet-pepper"

# Context: The robot is holding a SKYSCRAPER.
# But it really wants to make a salad.
initial_state = {
    "(agent-at lab)",

    # The Absurd Constraint:
    "(holding skyscraper)",

    # The Target:
    "(at sweet-pepper lab)",
    "(whole sweet-pepper)",
    "(clear sweet-pepper)",

    # The Tool (Available, but hands are full of building)
    "(at knife lab)",
    "(clear knife)"
}

goal_state = {
    "(cut-into-pieces sweet-pepper)"
}

print(f"\n{'='*80}\nScenario: {test_name}\n{'='*80}")

changes = plan_generator(obj_name, initial_state, goal_state)

if changes:
    print_plan_execution(changes, obj_name, initial_state)
    print(f"✓ PASS: {test_name}")
else:
    print(f"✗ FAIL: {test_name}")


Scenario: The Hungry Architect (Depth 3)

A* search starting
Found plan! Iterations: 16, Plan length: 3

📋 EXECUTION TRACE: SWEET-PEPPER
STEP   | ACTION                              | STATE CHANGES / INITIAL STATE           
------------------------------------------------------------------------------------------
0      | (INITIAL STATE)                     | (agent-at lab), (at knife lab), (at sweet-pepper lab), (clear knife), (clear sweet-pepper), (holding skyscraper), (whole sweet-pepper)
1      | (put-down skyscraper lab)           | ++ (at skyscraper lab), (clear skyscraper), (hand-empty) | -- (holding skyscraper)
2      | (pick-up knife lab)                 | ++ (holding knife) | -- (at knife lab), (hand-empty)
3      | (slice-object sweet-pepper lab)     | ++ (cut-into-pieces sweet-pepper) | -- (whole sweet-pepper)
------------------------------------------------------------------------------------------
✓ GOAL ACHIEVED

✓ PASS: The Hungry Architect (Depth 3)


In [11]:
# !!! WARNING !!! Using BFS, this will take LOOONG to run.
# You'll need to implement A* search (or something similar using a heuristic)

test_name = "Jurassic Park Protocol (Depth 4)"
obj_name = "dinosaur"

# Context: A dinosaur has escaped!
# The agent is in the lab, but the asset is outdoors.
initial_state = {
    "(agent-at lab)",
    "(hand-empty)",

    # The Asset (Loose in the wild)
    "(at dinosaur outdoors)",
    "(whole dinosaur)",
    "(clear dinosaur)"
}

# Goal: The dinosaur must be secured INSIDE the lab.
# (Merely holding it isn't enough; it must be 'at' the lab)
goal_state = {
    "(at dinosaur lab)"
}

print(f"\n{'='*80}\nScenario: {test_name}\n{'='*80}")

changes = plan_generator(obj_name, initial_state, goal_state)

if changes:
    print_plan_execution(changes, obj_name, initial_state)
    print(f"✓ PASS: {test_name}")
else:
    print(f"✗ FAIL: {test_name}")


Scenario: Jurassic Park Protocol (Depth 4)

A* search starting
Found plan! Iterations: 5, Plan length: 4

📋 EXECUTION TRACE: DINOSAUR
STEP   | ACTION                              | STATE CHANGES / INITIAL STATE           
------------------------------------------------------------------------------------------
0      | (INITIAL STATE)                     | (agent-at lab), (at dinosaur outdoors), (clear dinosaur), (hand-empty), (whole dinosaur)
1      | (walk-between-rooms lab outdoors)   | ++ (agent-at outdoors) | -- (agent-at lab)
2      | (pick-up dinosaur outdoors)         | ++ (holding dinosaur) | -- (at dinosaur outdoors), (hand-empty)
3      | (walk-between-rooms outdoors lab)   | ++ (agent-at lab) | -- (agent-at outdoors)
4      | (put-down dinosaur lab)             | ++ (at dinosaur lab), (clear dinosaur), (hand-empty) | -- (holding dinosaur)
------------------------------------------------------------------------------------------
✓ GOAL ACHIEVED

✓ PASS: Jurassic Park Proto